In [ ]:
### Once we are able to clean up and parse the data in good format, I can package the code into a package and publish it. ###

In [81]:
import os
import pandas as pd
import numpy as np
import requests as re
import json

apiKey_googleCivic = os.environ["apiKey_googleCivic"]

In [93]:
## Query from Google Civic information API

queryParam = {"key": apiKey_googleCivic,
              "address": "12955 E Cornell Ave. Aurora CO, 80014",
              "levels": ["country", "administrativeArea1"],
              "roles": ["legislatorLowerBody", "legislatorUpperBody"]}

## Response object
response = re.get("https://www.googleapis.com/civicinfo/v2/representatives", params=queryParam)

In [86]:
## What the response object look like

# print(response.encoding)  # UTF-8 encoding
# print(response.json())  # dict

## Prettify and serializes the json
# dump = (json.dumps(response.json(), indent=4, sort_keys=False))
# print(type(dump))  # str

UTF-8


In [95]:
## Description of the nested dict structure

response.json().keys()  # High level dictionary keys
response.json()["offices"]  # Can access nested dictionary or list of dict through selectors

[{'name': 'U.S. Senator',
  'divisionId': 'ocd-division/country:us/state:co',
  'levels': ['country'],
  'roles': ['legislatorUpperBody'],
  'officialIndices': [0, 1]},
 {'name': 'U.S. Representative',
  'divisionId': 'ocd-division/country:us/state:co/cd:6',
  'levels': ['country'],
  'roles': ['legislatorLowerBody'],
  'officialIndices': [2]},
 {'name': 'CO State Senator',
  'divisionId': 'ocd-division/country:us/state:co/sldu:26',
  'levels': ['administrativeArea1'],
  'roles': ['legislatorUpperBody'],
  'officialIndices': [3]},
 {'name': 'CO State Representative',
  'divisionId': 'ocd-division/country:us/state:co/sldl:41',
  'levels': ['administrativeArea1'],
  'roles': ['legislatorLowerBody'],
  'officialIndices': [4]}]

In [96]:
# display(response.json()["officials"])

# df_offices = pd.json_normalize(response.json()["offices"])
# display(df_offices)
# df_officials = pd.json_normalize(response.json()["officials"])
# display(df_officials)

df_offices = pd.DataFrame.from_dict(response.json()["offices"])
display(df_offices)
df_officials = pd.DataFrame.from_dict(response.json()["officials"])
display(df_officials)


print(type(df_officials.loc[0, "address"]))

,name,divisionId,levels,roles,officialIndices
0,U.S. Senator,ocd-division/country:us/state:co,[country],[legislatorUpperBody],"[0, 1]"
1,U.S. Representative,ocd-division/country:us/state:co/cd:6,[country],[legislatorLowerBody],[2]
2,CO State Senator,ocd-division/country:us/state:co/sldu:26,[administrativeArea1],[legislatorUpperBody],[3]
3,CO State Representative,ocd-division/country:us/state:co/sldl:41,[administrativeArea1],[legislatorLowerBody],[4]


,name,address,party,phones,urls,photoUrl,channels,geocodingSummaries,emails
0,John W. Hickenlooper,"[{'line1': 'B85', 'city': 'Washington', 'state...",Democratic Party,[(202) 224-5941],"[https://www.hickenlooper.senate.gov/, https:/...",http://www.colorado.gov/cms/governor/governor-...,"[{'type': 'Twitter', 'id': 'SenatorHick'}, {'t...",[{'queryString': 'B85 Russell Senate Office Bu...,NaN
1,Michael F. Bennet,[{'line1': '261 Russell Senate Office Building...,Democratic Party,[(202) 224-5852],"[https://www.bennet.senate.gov/, https://en.wi...",http://bioguide.congress.gov/bioguide/photo/B/...,"[{'type': 'Facebook', 'id': 'senbennetco'}, {'...",[{'queryString': '261 Russell Senate Office Bu...,NaN
2,Jason Crow,[{'line1': '1229 Longworth House Office Buildi...,Democratic Party,[(202) 225-7882],"[https://crow.house.gov/, https://en.wikipedia...",NaN,"[{'type': 'Facebook', 'id': 'RepJasonCrow'}, {...",[{'queryString': '1229 Longworth House Office ...,NaN
3,Jeff Bridges,"[{'line1': '200 East Colfax Avenue', 'city': '...",Democratic Party,[(303) 866-4846],[https://leg.colorado.gov/legislators/jeff-bri...,NaN,"[{'type': 'Facebook', 'id': 'bridgesforcolorad...","[{'queryString': '200 E COLFAX AVE STE 346, DE...",[jeff.bridges.senate@state.co.us]
4,Iman Jodeh,"[{'line1': '200 East Colfax Avenue', 'city': '...",Democratic Party,[(303) 866-2919],[https://leg.colorado.gov/legislators/iman-jod...,NaN,"[{'type': 'Facebook', 'id': 'ImanforColorado'}...","[{'queryString': '200 E COLFAX AVE STE 307, DE...",[iman.jodeh.house@state.co.us]


<class 'list'>


In [67]:
### Diff between `pd.json_normalize()` vs `pd.DataFrame.from_dict()` ### 

## json_normalize method
## - This method takes in a dictionary with value as a list 
## - The dictionaries in the list has column headers as dictionary keys
## - This wants unserialized json object, which means that it cannot be of type string but need to be dict
df = pd.json_normalize(response.json()["offices"])
display(df)


## from_dict method
## - This method is more flexible, similar to the pd.read_json() method
## - Can explicitly specify that the unserialized data is of columsn orientation
df2 = pd.DataFrame.from_dict(response.json()["offices"], orient="columns")
# df2 = pd.DataFrame.from_dict(response.json()["offices"])  # Same result
display(df2)

,name,divisionId,levels,roles,officialIndices
0,CO State Representative,ocd-division/country:us/state:co/sldl:41,[administrativeArea1],[legislatorLowerBody],[0]


,name,divisionId,levels,roles,officialIndices
0,CO State Representative,ocd-division/country:us/state:co/sldl:41,[administrativeArea1],[legislatorLowerBody],[0]
